In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

True

In [11]:
import os

# Set environment variables
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_PROJECT'] = 'cortex'

# Get keys from the environment
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")

if langchain_api_key:
    os.environ['LANGCHAIN_API_KEY'] = langchain_api_key
else:
    raise ValueError("LANGCHAIN_API_KEY is not set in the environment.")

if groq_api_key:
    os.environ['GROQ_API_KEY'] = groq_api_key
else:
    raise ValueError("GROQ_API_KEY is not set in the environment.")

PART 5 - MULTI QUERY

In [12]:
import os
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

# Set USER_AGENT before making requests
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Load Documents
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split - Chunking
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50
)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Embed
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}

# Initialize HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

# Index with FAISS
vectorstore = FAISS.from_documents(documents=splits, embedding=hf_embeddings)

# Retrieve
retriever = vectorstore.as_retriever()

Multi-Query Prompt

In [13]:
from langchain.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

# Disable LangSmith tracing to prevent API errors
os.environ["LANGCHAIN_TRACING_V2"] = "false" 

# Define the template for generating multiple perspectives on the user's question
template = """
You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 

Provide these alternative questions separated by newlines.

Original question: {question}
"""

# Create the prompt template using ChatPromptTemplate
prompt_perspectives = ChatPromptTemplate.from_template(template)

# Define the pipeline for generating alternative questions
generate_queries = (
    prompt_perspectives  # Use the prompt template
    | ChatGroq(temperature=0)  # Use the ChatGroq model with zero temperature (deterministic responses)
    | StrOutputParser()  # Parse the output into a clean format (a single string with questions separated by newlines)
    | (lambda x: "\n".join([q.strip() for q in x.split("\n") if q.strip()]))  # Clean and join questions with newlines
)

# Example: Use the pipeline to generate alternative questions for a given user question
user_question = "What are the benefits of using LangChain for NLP tasks?"
alternative_questions = generate_queries.invoke({"question": user_question})

# Print the generated alternative questions
print(alternative_questions)

1. How does LangChain enhance natural language processing tasks?
2. In what ways does LangChain improve the efficiency of NLP projects?
3. Can you list the advantages of implementing LangChain in NLP workflows?
4. What are the key benefits that LangChain brings to the table for NLP applications?
5. How can LangChain contribute to the success of natural language processing tasks?


In [14]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

15

In [15]:
from langchain.load import dumps, loads
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# RAG template
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGroq(temperature=0)

# Define the final RAG chain
final_rag_chain = (
    {"context": retrieval_chain, "question": itemgetter("question")}  # Ensure the mapping is correct
    | prompt
    | llm
    | StrOutputParser()
    | (lambda x: "\n".join([q.strip() for q in x.split("\n") if q.strip()]))
)

# Define the question
question = "What is task decomposition for LLM agents?"

# Invoke the chain with the question and get the result
result = final_rag_chain.invoke({"question": question})

# Print the result
print(result)

Task decomposition for LLM agents refers to the process of breaking down a complex task into smaller, manageable subtasks that can be executed by a large language model (LLM) agent. This allows the agent to handle complex tasks by focusing on one subtask at a time, making it more efficient and effective. The subtasks are often defined in a specific format, such as the ReAct prompt template, which includes explicit steps for the LLM to think, act, and observe. The agent repeats this process until the task is completed. Task decomposition is an important aspect of building LLM-centered agents, as it helps to address common limitations such as hallucination and inefficient planning.


PART 6 - RAG-FUSION

In [16]:
from langchain.prompts import ChatPromptTemplate

# RAG-Fusion: Related
template = """
You are a helpful assistant tasked with generating multiple search queries based on a single input query. 
The goal is to create queries that are semantically related to the input while capturing different aspects of the topic.

Input Query: {question}

Output: Provide exactly 4 related search queries, each on a new line.
"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# Define the query generation pipeline
generate_queries = (
    prompt_rag_fusion
    | ChatGroq(temperature=0)
    | StrOutputParser()
    | (lambda x: [
        query.strip().lstrip("1234567890. ")  # Remove numbering and extra spaces
        for query in x.split("\n") if query.strip()  # Remove empty lines
    ])
)


In [18]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    """Applies Reciprocal Rank Fusion (RRF) to combine ranked document lists."""
    fused_scores = {}

    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)  # Serialize document for use as dictionary key
            fused_scores[doc_str] = fused_scores.get(doc_str, 0) + 1 / (rank + k)

    # Sort by score in descending order and deserialize documents
    return [
        (loads(doc), score) for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

# RAG Fusion Retrieval Chain
retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion

# Invoke the chain with a question
docs = retrieval_chain_rag_fusion.invoke({"question": question})

# Output the length of the fused document list
print(f"Number of retrieved documents: {len(docs)}")


Number of retrieved documents: 9


In [19]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
    | (lambda x: "\n".join([line.strip() for line in x.split("\n") if line.strip()]))
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM agents is the process of breaking down a complex task into smaller and simpler steps. This can be done using techniques such as Chain of Thought (CoT) or Tree of Thoughts, where the model is instructed to "think step by step" and generate multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier or majority vote. Task decomposition can also be done using simple prompting, task-specific instructions, or human inputs.'

PART 7 - DECOMPOSITION

In [20]:
from langchain.prompts import ChatPromptTemplate

# Decomposition Template
template = """
You are a helpful assistant tasked with breaking down a complex input question into smaller, focused sub-questions. 
Each sub-question should address a specific aspect of the main question, allowing them to be answered in isolation.

Input: {question}

Your goal:
- Decompose the input question into exactly 3 relevant and concise sub-questions.
- Ensure each sub-question is clear, non-overlapping, and focused on a distinct aspect of the input.

Output:
1. [First sub-question]
2. [Second sub-question]
3. [Third sub-question]
"""
prompt_decomposition = ChatPromptTemplate.from_template(template)


In [21]:
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatGroq(temperature=0)

# Chain for generating sub-questions via decomposition
generate_queries_decomposition = (
    prompt_decomposition  # The prompt template
    | llm  # Language model to process the prompt
    | StrOutputParser()  # Parses the LLM output to a string
    | (lambda x: [q.strip() for q in x.split("\n") if q.strip()])  # Clean and split into a list of sub-questions
)

# Input Question
question = "What are the main components of an LLM-powered autonomous agent system?"

# Run the Chain
try:
    questions = generate_queries_decomposition.invoke({"question": question})
    # Print the generated sub-questions
    print("\n".join(questions))
except Exception as e:
    print(f"An error occurred: {e}")

1. What are the key components of a Language Model (LLM) that contribute to its use in an autonomous agent system?
2. How does an autonomous agent system incorporate an LLM for decision-making and task execution?
3. What are the main supporting systems or components required for an LLM-powered autonomous agent to function effectively?


In [22]:
template = """
You are a helpful assistant tasked with answering a given question using the provided background question-answer pairs and additional context.

Question:
{question}

Background Question-Answer Pairs:
{q_a_pairs}

Additional Context:
{context}

Using the above information, provide a clear and concise answer to the question: {question}
"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [23]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser

def format_qa_pair(question, answer):
    """Format a question and answer pair as a string."""
    return f"Question: {question}\nAnswer: {answer}\n"

# Initialize the language model (LLM)
llm = ChatGroq(temperature=0)

# Initialize Q&A pairs as an empty string
q_a_pairs = ""

# Loop through the list of questions and process each one
for q in questions:
    # Define the RAG chain for retrieving and answering
    rag_chain = (
        {
            "context": itemgetter("question") | retriever,  # Retrieve context for the current question
            "question": itemgetter("question"),             # Extract the current question
            "q_a_pairs": itemgetter("q_a_pairs")            # Include existing Q&A pairs
        }
        | decomposition_prompt  # Apply the decomposition prompt
        | llm                   # Use the LLM to generate an answer
        | StrOutputParser()     # Parse the output into a clean string
    )

    # Generate an answer for the current question
    answer = rag_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})

    # Format the question and answer pair
    q_a_pair = format_qa_pair(q, answer)

    # Append the new Q&A pair to the existing pairs
    q_a_pairs += f"\n---\n{q_a_pair.strip()}"

# Print the final Q&A pairs (optional)
print(q_a_pairs.strip())

---
Question: 1. What are the key components of a Language Model (LLM) that contribute to its use in an autonomous agent system?
Answer: The key components of a Language Model (LLM) that contribute to its use in an autonomous agent system include Planning, Memory, and the ability to call external symbolic tools. Planning involves subgoal and decomposition, as well as reflection and refinement. Memory allows the agent to store and retrieve past information. The LLM's capability to determine when and how to use external symbolic tools, such as a calculator or API, is also crucial for its effective use in an autonomous agent system.
---
Question: 2. How does an autonomous agent system incorporate an LLM for decision-making and task execution?
Answer: An autonomous agent system incorporates a Language Model (LLM) for decision-making and task execution by utilizing its key components, which include Planning, Memory, and the ability to call external symbolic tools. The LLM can decompose comp

Answer Individually

In [ ]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# Pull the RAG prompt template from the hub
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question, prompt_rag, sub_question_generator_chain):
    """
    Perform RAG (Retrieve and Generate) for each sub-question.
    
    Args:
        question (str): The main input question.
        prompt_rag: The RAG prompt template.
        sub_question_generator_chain: A chain that generates sub-questions.
        retriever: A retriever object to fetch relevant documents.
        llm: The language model for answering questions.

    Returns:
        tuple: A list of answers and the corresponding sub-questions.
    """
    # Generate sub-questions from the input question
    sub_questions = sub_question_generator_chain.invoke({"question": question})

    # Initialize a list to store answers for each sub-question
    rag_results = []

    # Process each sub-question
    for sub_question in sub_questions:
        # Retrieve documents relevant to the sub-question
        rretrieved_docs = retriever.invoke(sub_question)

        # Generate an answer using the RAG prompt, LLM, and retrieved documents
        answer = (
            prompt_rag  # Apply the RAG prompt
            | llm       # Pass it through the language model
            | StrOutputParser()  # Parse the LLM output into a clean string
        ).invoke({"context": retrieved_docs, "question": sub_question})

        # Append the generated answer to the results list
        rag_results.append(answer)

    return rag_results, sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

/var/folders/bg/7gm4yt417px42myyczdlqf1h0000gn/T/ipykernel_53172/4103660154.py:33: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(sub_question)


In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

def format_qa_pairs(questions, answers):
    """
    Format a list of questions and their corresponding answers into a structured string.

    Args:
        questions (list): List of questions.
        answers (list): List of answers corresponding to the questions.

    Returns:
        str: Formatted string of Q&A pairs.
    """
    return "\n\n".join(
        f"Question {i}: {q}\nAnswer {i}: {a}" for i, (q, a) in enumerate(zip(questions, answers), start=1)
    ).strip()

# Format the Q&A pairs to create the context
context = format_qa_pairs(questions, answers)

# Define the prompt template for synthesizing an answer
template = """
Here is a set of Q+A pairs:

{context}

Use these to synthesize a clear and concise answer to the following question:

{question}
"""
prompt = ChatPromptTemplate.from_template(template)

# Define the RAG chain
final_rag_chain = (
    prompt  # Use the prompt template
    | llm  # Pass the input through the language model
    | StrOutputParser()  # Parse the output into a clean string
)

# Invoke the chain with the given context and question
result = final_rag_chain.invoke({"context": context, "question": question})

# Output the result
print(result)


The main components of a LLM-powered autonomous agent system include Planning, Memory, Reflection and Refinement, and an optional external classical planner. Planning is responsible for breaking down large tasks into smaller, manageable subgoals and planning ahead. Memory aids in storing and retrieving information for the agent. Reflection and Refinement enable the agent to learn from past actions, critique them, and refine future steps. An external classical planner can be used for long-horizon planning. These components work together to enable the agent to perceive, understand, make decisions, and execute actions in its environment.


PART 8 - STEP BACK

In [31]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

# Define few-shot examples
examples = [
    {
        "input": "Could the members of The Police perform lawful arrests?",
        "output": "What can the members of The Police do?",
    },
    {
        "input": "Jan Sindel was born in what country?",
        "output": "What is Jan Sindel’s personal history?",
    },
]

# Transform examples into example messages using a message prompt template
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# Create a few-shot prompt template with the example prompt and examples
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# Combine the system message, few-shot examples, and user question into the final prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert in world knowledge. Your task is to reframe a specific question into a more generic, step-back question that is easier to answer. Here are a few examples:""",
        ),
        # Add few-shot examples
        few_shot_prompt,
        # Add the user question
        ("user", "{question}"),
    ]
)


In [32]:
generate_queries_step_back = prompt | ChatGroq(temperature=0) | StrOutputParser()
question = "What is task decomposition for LLM agents?"
generate_queries_step_back.invoke({"question": question})

'How can a LLM agent break down a task into smaller, manageable parts?'

In [34]:
# Define the response prompt template
response_prompt_template = """
You are an expert in world knowledge. I will ask you a question. Your response should be comprehensive and aligned with the provided contexts if they are relevant. Otherwise, ignore them if they are not relevant.

# Normal Context:
{normal_context}

# Step-Back Context:
{step_back_context}

# Original Question:
{question}

# Answer:
"""

# Create the response prompt
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

# Define the chain for context retrieval and response generation
chain = (
    {
        # Retrieve normal context using the original question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve step-back context using the step-back question
        "step_back_context": generate_queries_step_back | retriever,
        # Pass through the original question
        "question": lambda x: x["question"],
    }
    | response_prompt  # Generate a prompt with the retrieved context and question
    | ChatGroq(temperature=0)  # Generate the response using the LLM
    | StrOutputParser()  # Parse the response into a clean string
)

# Invoke the chain with the input question
result = chain.invoke({"question": question})

# Print the generated response
print(result)


Task decomposition for LLM (Latent Dirichlet Allocation) agents refers to the process of breaking down a complex task into smaller, manageable subtasks. This allows the agent to plan and execute the task more effectively. There are several methods for task decomposition in LLM agents:

1. Chain of Thought (CoT): This is a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to "think step by step" to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and sheds light on the interpretation of the model's thinking process.
2. Tree of Thoughts: This extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state eval

PART 9 - HyDE

In [35]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# Define the HyDE document generation template
template = """
Please write a scientific paper passage to answer the following question:

Question: {question}

Passage:
"""

# Create a prompt template using ChatPromptTemplate
prompt_hyde = ChatPromptTemplate.from_template(template)

# Define the document generation chain
generate_docs_for_retrieval = (
    prompt_hyde  # Use the HyDE prompt template
    | ChatGroq(temperature=0)  # Generate the document using the language model
    | StrOutputParser()  # Parse the LLM output into a clean string
)

# Define the question
question = "What is task decomposition for LLM agents?"

# Invoke the chain to generate the passage
generated_passage = generate_docs_for_retrieval.invoke({"question": question})

# Print the generated passage
print(generated_passage)


Task decomposition is a crucial aspect of building large language models (LLMs) that can effectively perform complex tasks. It involves breaking down a complex task into smaller, manageable subtasks that the LLM can handle more efficiently. This process is similar to the way humans approach complex problems by dividing them into smaller, more manageable parts.

In the context of LLMs, task decomposition enables the model to focus on specific aspects of a task, reducing the cognitive load and improving the overall performance. For example, if the task is to write a scientific paper, the LLM can be decomposed into subtasks such as:

1. Generating an outline for the paper
2. Conducting literature reviews and summarizing relevant sources
3. Developing arguments and evidence to support the paper's thesis
4. Writing the introduction, body, and conclusion of the paper
5. Formatting the paper according to the required style

By breaking down the task in this way, the LLM can focus on one subta

In [36]:
# Retrieve
retrieval_chain = generate_docs_for_retrieval | retriever 
retrieved_docs = retrieval_chain.invoke({"question":question})
retrieved_docs

[Document(id='bce32ac7-6491-4ee1-b8d6-a70459a5f1af', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The sear

In [37]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# Define the RAG prompt template
template = """
Answer the following question based on the provided context:

Context:
{context}

Question:
{question}
"""

# Create a prompt from the template
prompt = ChatPromptTemplate.from_template(template)

# Define the RAG chain
final_rag_chain = (
    prompt  # Generate the prompt using the template
    | ChatGroq(temperature=0)  # Use the LLM to process the prompt
    | StrOutputParser()  # Parse the output into a clean string
)

# Invoke the RAG chain with the context and question
result = final_rag_chain.invoke({
    "context": retrieved_docs,  # The retrieved documents as context
    "question": question  # The question to be answered
})

# Print the result
print(result)

Task decomposition for LLM (Latent Dirichlet Allocation) agents refers to the process of breaking down a complex task into smaller, manageable subtasks. This can be done using simple prompting techniques, task-specific instructions, or human inputs. The 'Chain of Thought' (CoT) technique, as described in the document, is a standard prompting technique for enhancing model performance on complex tasks. It instructs the model to 'think step by step' to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. The 'Tree of Thoughts' (ToT) extends CoT by exploring multiple reasoning possibilities at each step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.
